<a href="https://colab.research.google.com/github/aledelorbe/pt2Codigos/blob/main/SegundasActividades/Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Cargar librerias

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
from sklearn.cluster import KMeans

### Cargar los dataFrame

In [7]:
nuevaMortalidadLimpia = '/content/drive/MyDrive/Pt2/Primeras Actividades/0 Bases de Datos/Nueva Mortalidad Limpia/'

df2010 = pd.read_csv(nuevaMortalidadLimpia + '2010.csv')
df2011 = pd.read_csv(nuevaMortalidadLimpia + '2011.csv')
df2012 = pd.read_csv(nuevaMortalidadLimpia + '2012.csv')
df2013 = pd.read_csv(nuevaMortalidadLimpia + '2013.csv')
df2014 = pd.read_csv(nuevaMortalidadLimpia + '2014.csv')
df2015 = pd.read_csv(nuevaMortalidadLimpia + '2015.csv')
df2016 = pd.read_csv(nuevaMortalidadLimpia + '2016.csv')
df2017 = pd.read_csv(nuevaMortalidadLimpia + '2017.csv')
df2018 = pd.read_csv(nuevaMortalidadLimpia + '2018.csv')
df2019 = pd.read_csv(nuevaMortalidadLimpia + '2019.csv')

### Unir todos los fragmentos

In [8]:
dfDefinitivo = pd.concat([df2010, df2011, df2012, df2013, df2014, df2015, df2016, df2017, df2018, df2019], ignore_index=True)

### Determinacion de valores distintos en cada columna

In [9]:
# Definicion de la funcion que imprimira los valores distintos de determinada columna en orden alfabetico
def imprimirValoresDistintos(df20xx, nombreColumna):
  titulo = f"Columna {nombreColumna}".center(100, '-')
  valoresDistintos = list(df2010[nombreColumna].unique())
  valoresDistintos.sort()
  print(titulo)
  print(valoresDistintos)
  print(f'Cantidad de elementos: {len(valoresDistintos)}')

  return valoresDistintos

estados = imprimirValoresDistintos(dfDefinitivo, 'ent_resid_nm')
canceres = imprimirValoresDistintos(dfDefinitivo, 'Organ')
trabajos = imprimirValoresDistintos(dfDefinitivo, 'desc_ocupacion')

----------------------------------------Columna ent_resid_nm----------------------------------------
['Aguascalientes', 'Baja California', 'Baja California Sur', 'Campeche', 'Chiapas', 'Chihuahua', 'Coahuila de Zaragoza', 'Colima', 'Distrito Federal', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'Michoacán de Ocampo', 'Morelos', 'México', 'Nayarit', 'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo', 'San Luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz de Ignacio de la Llave', 'Yucatán', 'Zacatecas']
Cantidad de elementos: 32
-------------------------------------------Columna Organ--------------------------------------------
['Bladder', 'Bone', 'Brain', 'Breast', 'Cervix Uteri', 'Colon and Rectum', 'Esophagus', 'Eyes', 'Female Genitals', 'Gallbladder', 'Heart', "Hodgkin's Lymphoma", 'Kidney', 'Larynx', 'Leukemia', 'Liver ', 'Lungs', 'Lymphoma', 'Male Genitales', 'Melanoma', 'Mesothelioma', 'Mylenoma', 'Nervous', 'Oral Cavity an

In [10]:
# Para este caso se hace distinto ya que se desea crear una jerarquia (de menor a mayor jerarquia)
nivelesEscolares = [
    'No especificada ',
    'No aplica a menores de 3 años',
    'Sin escolaridad',
    'Preescolar',
    'Primaria completa',
    'Secundaria incompleta',
    'Secundaria completa',
    'Bachillerato o preparatoria',
    'Profesional',
    'Posgrado',
]

In [11]:
# Para contar cuantas personas hay con cada tipo de cancer en cierto estado
# contemplando que puede haber 0 personas con cierto tipo de cancer en cierto estado
def contadorFrecuencias(df, estado, variable, catalogoX):
  result = None
  for elemento in catalogoX:
    condicionX = (df['ent_resid_nm'] == estado) & (df[variable] == elemento)
    resultAux = df[condicionX].groupby(variable).size().reset_index(name='Frecuencia')

    if len(resultAux) == 0:
      dfAux = pd.DataFrame({variable: [elemento], 'Frecuencia': [0]})
      result = pd.concat([result, resultAux, dfAux], ignore_index=True)
    else:
      result = pd.concat([result, resultAux], ignore_index=True)
  print(result)

  return result

# Realizar una especie de transpuesta para pasar de tener filas asi: 'tipoCancer1', 'frecuencia1'; tipoCancer2', 'frecuencia2';
# tener columnas asi: 'tipoCancer1', 'frecuencia1', tipoCancer2', 'frecuencia2',
def transpuesta(df, variable):
  dfTranspuesta = pd.DataFrame({"vacio": [0]})
  for index, row in df.iterrows():
    dfTranspuesta[variable + str(index)] = row[variable]
    dfTranspuesta['cant' + str(index)] = row["Frecuencia"]
  dfTranspuesta = dfTranspuesta.drop(['vacio'], axis=1)
  print(dfTranspuesta)

  return dfTranspuesta

# Para unir dos dataframe (uno a lado del otro).
def unirDataframes(df1, df2):
    # Crear columnas 'id' en ambos dataframes
    df1['id1'] = 1
    df2['id2'] = 1

    # Realizar union y eliminar las columnas 'id' de ambos dataframes
    df_union = pd.merge(df1, df2, left_on='id1', right_on='id2')
    df_union = df_union.drop(['id1', 'id2'], axis=1)

    return df_union

def asociador(estado, canceres, trabajos, nivelesEscolares):
  titulo = f"Estado {estado}".center(100, '-')
  print(titulo)

  dfEstadoX = pd.DataFrame({'estado': [estado]})

  # Asociar los tipos de canceres al estadoX
  dfFrec = contadorFrecuencias(dfDefinitivo, estado, 'Organ', canceres)
  dfCancer = transpuesta(dfFrec, 'Organ')

  # Asociar los diferentes trabajos al estadoX
  dfFrec = contadorFrecuencias(dfDefinitivo, estado, 'desc_ocupacion', trabajos)
  dfTrabajo = transpuesta(dfFrec, 'desc_ocupacion')

  # Asociar los diferentes niveles academicos al estadoX
  dfFrec = contadorFrecuencias(dfDefinitivo, estado, 'desc_escolaridad', nivelesEscolares)
  dfEducacion = transpuesta(dfFrec, 'desc_escolaridad')

  # Aqui hace falta unir los df's: cancer, trabajo y educacion
  #dfEstadoX = unirDataframes(dfEstadoX, dfCancer)



#estadosCancer = []
#for estado in estados:
 # asociador(estado, 'Organ')

asociador('Aguascalientes', canceres, trabajos, nivelesEscolares)


---------------------------------------Estado Aguascalientes----------------------------------------
                      Organ  Frecuencia
0                   Bladder         123
1                      Bone         130
2                     Brain         276
3                    Breast         710
4              Cervix Uteri         410
5          Colon and Rectum         564
6                 Esophagus          97
7                      Eyes           7
8           Female Genitals          16
9               Gallbladder         250
10                    Heart           2
11       Hodgkin's Lymphoma          51
12                   Kidney         219
13                   Larynx          61
14                 Leukemia         473
15                   Liver          366
16                    Lungs         865
17                 Lymphoma         303
18           Male Genitales          14
19                 Melanoma          75
20             Mesothelioma          23
21                 



### Transformaciones

#### Educación

In [18]:
# Definicion de la funcion que codificara a la variable 'desc_escolaridad'
def codificadorOrdinalEducacion(df20xx, nombreDf, educaciones):
  print(df20xx)

  # Crear cada elemento de 'educaciones' como una columna y rellenarla con puros 0's
  for elemento in educaciones:
    df20xx[elemento] = 0

  # Hacer que: si un registro tiene cierta categoria, todas las categorias de menor jerarquia a esa, se les coloque
  # un uno. Pe si un registro tiene la categoria primaria completa a esa columna colocarle un uno, pero tambien
  # en 'preescolar', en 'No aplica a menores de 3 años' y en 'No especificada ' ó si una persona tiene el valor de
  # posgrado tendra un uno en esa columna y en todas las demas (porque es el de mayor jerarquia).
  condicion1 = (df20xx['desc_escolaridad'] == 'No especificada ')
  df20xx.loc[condicion1, educaciones[0]] = 1

  condicion2 = (df20xx['desc_escolaridad'] == 'No aplica a menores de 3 años')
  df20xx.loc[condicion2, educaciones[0:2]] = 1

  condicion3 = (df20xx['desc_escolaridad'] == 'Sin escolaridad')
  df20xx.loc[condicion3, educaciones[0:3]] = 1

  condicion4 = (df20xx['desc_escolaridad'] == 'Preescolar')
  df20xx.loc[condicion4, educaciones[0:4]] = 1

  condicion5 = (df20xx['desc_escolaridad'] == 'Primaria completa')
  df20xx.loc[condicion5, educaciones[0:5]] = 1

  condicion6 = (df20xx['desc_escolaridad'] == 'Secundaria incompleta')
  df20xx.loc[condicion6, educaciones[0:6]] = 1

  condicion7 = (df20xx['desc_escolaridad'] == 'Secundaria completa')
  df20xx.loc[condicion7, educaciones[0:7]] = 1

  condicion8 = (df20xx['desc_escolaridad'] == 'Bachillerato o preparatoria')
  df20xx.loc[condicion8, educaciones[0:8]] = 1

  condicion9 = (df20xx['desc_escolaridad'] == 'Profesional')
  df20xx.loc[condicion9, educaciones[0:9]] = 1

  condicion10 = (df20xx['desc_escolaridad'] == 'Posgrado')
  df20xx.loc[condicion10, educaciones[0:10]] = 1

  # Imprimir resultado
  titulo = f"DataFrame {nombreDf}".center(100, '-')
  print(titulo)
  print(df20xx.to_string())

  return df20xx

# Crear df con los niveles escolares
dfEducacionSinCod = pd.DataFrame({'desc_escolaridad': nivelesEscolares})

# Mandar a llamar a la funcion dumizadora de la variable 'escolaridad'
dfEducacionCod = codificadorOrdinalEducacion(dfEducacionSinCod, 'Niveles escolares codificados', nivelesEscolares)

                desc_escolaridad
0               No especificada 
1  No aplica a menores de 3 años
2                Sin escolaridad
3                     Preescolar
4              Primaria completa
5          Secundaria incompleta
6            Secundaria completa
7    Bachillerato o preparatoria
8                    Profesional
9                       Posgrado
------------------------------DataFrame Niveles escolares codificados-------------------------------
                desc_escolaridad  No especificada   No aplica a menores de 3 años  Sin escolaridad  Preescolar  Primaria completa  Secundaria incompleta  Secundaria completa  Bachillerato o preparatoria  Profesional  Posgrado
0               No especificada                  1                              0                0           0                  0                      0                    0                            0            0         0
1  No aplica a menores de 3 años                 1                              1   

#### Empleo

In [21]:
# Definicion de la funcion que dumizara distintas variables en cada fragmento
def duminizador(df20xx, nombreDf, nombreVariable):
  print(df20xx)

  # Dumizar variable
  dummies = pd.get_dummies(df20xx[nombreVariable], drop_first = True)
  # drop_first = True -> es para que nos agregue solo k-1 categorias distintas y no se agregue la categoria k la cual solo provoca redundancia
  df20xx = pd.concat([df20xx, dummies], axis = 1) # Añadimos las variables binarias al DataFrame

  # Solicitud de las categorias que se encuentran en la variable: 'desc_ocupacion'
  nombresCategoriasX = dummies.columns.tolist()

  # Imprimir resultado
  titulo = f"DataFrame {nombreDf}".center(100, '-')
  print(titulo)
  print(df20xx.to_string())

  return df20xx

# Crear df con los tipos de trabajo
dfTrabajosSinCod = pd.DataFrame({'desc_ocupacion': trabajos})

# Mandar a llamar a la funcion dumizadora de la variable 'trabajos'
dfTrabajosCod = duminizador(dfTrabajosSinCod, 'Trabajos codificados', 'desc_ocupacion')

                                       desc_ocupacion
0   Comerciantes, empleados en ventas y agentes de...
1                    Funcionarios, directores y jefes
2                      Insuficientemente especificada
3                       No aplica a menores de 5 años
4                                     No especificada
5                                          No trabaja
6   Operadores de maquinaria industrial, ensamblad...
7                           Profesionistas y técnicos
8                            Trabajadores artesanales
9   Trabajadores auxiliares en actividades adminis...
10  Trabajadores en actividades agrícolas, ganader...
11  Trabajadores en actividades elementales y de a...
12  Trabajadores en servicios personales y vigilancia
-----------------------------------DataFrame Trabajos codificados-----------------------------------
                                                                              desc_ocupacion  Funcionarios, directores y jefes  Insuficientemen

#### Cancer

In [23]:
# Crear df con los tipos de cancer
dfCanceresSinCod = pd.DataFrame({'Organ': canceres})

# Mandar a llamar a la funcion dumizadora de la variable 'trabajos'
dfCanceresCod = duminizador(dfCanceresSinCod, 'Canceres codificados', 'Organ')

                      Organ
0                   Bladder
1                      Bone
2                     Brain
3                    Breast
4              Cervix Uteri
5          Colon and Rectum
6                 Esophagus
7                      Eyes
8           Female Genitals
9               Gallbladder
10                    Heart
11       Hodgkin's Lymphoma
12                   Kidney
13                   Larynx
14                 Leukemia
15                   Liver 
16                    Lungs
17                 Lymphoma
18           Male Genitales
19                 Melanoma
20             Mesothelioma
21                 Mylenoma
22                  Nervous
23  Oral Cavity and Pharynx
24       Outter Respiratory
25                    Ovary
26                 Pancreas
27                 Prostate
28       Skin non Melanoma 
29              Soft Tissue
30                  Stomach
31                   Testis
32                  Thyroid
33                  Trachea
34                  

#### Entidad Federativa

In [24]:
# Crear df con los tipos de cancer
dfEstadosSinCod = pd.DataFrame({'ent_resid_nm': estados})

# Mandar a llamar a la funcion dumizadora de la variable 'trabajos'
dfEstadosCod = duminizador(dfEstadosSinCod, 'Estados codificados', 'ent_resid_nm')

                       ent_resid_nm
0                    Aguascalientes
1                   Baja California
2               Baja California Sur
3                          Campeche
4                           Chiapas
5                         Chihuahua
6              Coahuila de Zaragoza
7                            Colima
8                  Distrito Federal
9                           Durango
10                       Guanajuato
11                         Guerrero
12                          Hidalgo
13                          Jalisco
14              Michoacán de Ocampo
15                          Morelos
16                           México
17                          Nayarit
18                       Nuevo León
19                           Oaxaca
20                           Puebla
21                        Querétaro
22                     Quintana Roo
23                  San Luis Potosí
24                          Sinaloa
25                           Sonora
26                          

### Algoritmos/Modelos